In [2]:
'''
    Author - Oyesh Mann Singh
    Date - 11/30/2018
    Description 
        - Usage of Word Embedding
        - Deep Learning
'''

import os
import sys
import time
import re
import string
import unicodecsv as csv
import unicodedata as un
import pandas as pd
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize 
from gensim.models import Word2Vec, KeyedVectors

In [2]:
'''
    Read all files into one corpus file
    Break down each sentence into new line
    Remove punctuation except selective punctuation
    Lowercase
    Replace numbers with <num>
    Do not remove stop words
    Remove '.'
'''
print()

### DO NOT RUN THIS SECTION TWICE

In [2]:
'''
    DO NOT RUN
'''
in_path = './data/aclImdb/data/'
out_file = './data/raw_corpus.txt'

# Create a corpus and remove html break 
with open(out_file, 'w', encoding='utf8') as out_f:
    for root, dirs, files in os.walk(in_path, topdown=True):
        if(len(files) > 0):
            for each_file in files:
                file_path = os.path.join(root, each_file)
                content = open(file_path, encoding='utf-8-sig').read()
                out_f.write(content)

out_f.close()    


In [4]:
out_file = './data/raw_corpus.txt'
in_f = open(out_file, encoding='utf-8').read()

In [5]:
# -*- coding: utf-8 -*-
# https://stackoverflow.com/a/31505798/4595807
'''
    Split reviews into sentences with special conditions
'''
import re
alphabets= "([A-Za-z])"
numbers = "([0-9])"
symbols = '[,\\\/@#$&%\-\'\(\):;=\+*\^\~\[\]\"\|]'
prefixes = "(Mr|St|Mrs|Ms|Dr)[.]"
suffixes = "(Inc|Ltd|Jr|Sr|Co)"
starters = "(Mr|Mrs|Ms|Dr|He\s|She\s|It\s|They\s|Their\s|Our\s|We\s|But\s|However\s|That\s|This\s|Wherever)"
acronyms = "([A-Z][.][A-Z][.](?:[A-Z][.])?)"
websites = "[.](com|net|org|io|gov)"

def split_into_sentences(text):
    text = " " + text + "  "
    text = text.replace("\n"," ")
    text = re.sub(prefixes,"\\1<prd>",text)
    text = re.sub(websites,"<prd>\\1",text)
    
    if "Ph.D" in text: text = text.replace("Ph.D.","Ph<prd>D<prd>")
    text = re.sub("\s" + alphabets + "[.] "," \\1<prd> ",text)
    text = re.sub(acronyms+" "+starters,"\\1<stop> \\2",text)
    text = re.sub(alphabets + "[.]" + alphabets + "[.]" + alphabets + "[.]","\\1<prd>\\2<prd>\\3<prd>",text)
    text = re.sub(alphabets + "[.]" + alphabets + "[.]","\\1<prd>\\2<prd>",text)
    text = re.sub(" "+suffixes+"[.] "+starters," \\1<stop> \\2",text)
    text = re.sub(" "+suffixes+"[.]"," \\1<prd>",text)
    text = re.sub(" " + alphabets + "[.]"," \\1<prd>",text)
    text = re.sub(numbers," <num>",text)
    
    

    if "”" in text: text = text.replace(".”","”.")
    if "\"" in text: text = text.replace(".\"","\".")
    if "!" in text: text = text.replace("!\"","\"!")
    if "?" in text: text = text.replace("?\"","\"?")
    text = text.replace(".","<stop>")
    text = text.replace("?","<stop>")
    text = text.replace("!","<stop>")
    text = text.replace("<prd>",".")
    text = text.replace("<br />","")
    
    text = re.sub(symbols,"",text)
    text = text.lower()
    sentences = text.split("<stop>")
    sentences = sentences[:-1]
    sentences = [s.strip() for s in sentences if len(s) > 1]
    return sentences

In [6]:
sent = split_into_sentences(in_f)

In [7]:
print(sent[0:2])

['once again mr. costner has dragged out a movie for far longer than necessary', 'aside from the terrific sea rescue sequences of which there are very few i just did not care about any of the characters']


In [8]:
'''
    Tokenize each sentence
'''
import nltk
total_sents = []

for each in sent:
    total_sents.append(each.split())

In [9]:
total_sents[0:2]

[['once',
  'again',
  'mr.',
  'costner',
  'has',
  'dragged',
  'out',
  'a',
  'movie',
  'for',
  'far',
  'longer',
  'than',
  'necessary'],
 ['aside',
  'from',
  'the',
  'terrific',
  'sea',
  'rescue',
  'sequences',
  'of',
  'which',
  'there',
  'are',
  'very',
  'few',
  'i',
  'just',
  'did',
  'not',
  'care',
  'about',
  'any',
  'of',
  'the',
  'characters']]

In [10]:
'''
    Creating word2vec embeddings
'''
from gensim.test.utils import get_tmpfile
from gensim.models import Word2Vec

path = get_tmpfile("./data/word2vec.model")

model = Word2Vec(total_sents, size=50, window=5, min_count=1, workers=4)
model.wv.save_word2vec_format('./data/word2vec.txt')